# Imports

In [67]:
import numpy as np
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim, utils
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random
import h5py

from sklearn import metrics
from matplotlib import pyplot as plt

import h5py
import numpy as np
import os #file_path = "your_file.h5"
import pandas as pd

# Section Requiring Modification (Dataset class and Model class)

In [ ]:
'''
MODIFY THIS CELL TO INCLUDE YOUR OWN DATASET
'''

class customDataset (torch.utils.data.Dataset):
    def __init__(self, path, file_names):
        super(customDataset, self).__init__()

        data1 = []
        data2 = []
        data3 = []
        if (type(file_names)!=list):
            file_names = [file_names]

        for file_name in file_names:
            with h5py.File(path + file_name, 'r') as f:
                dataset = (f["emg2qwerty"]["timeseries"][:])

            for i, j, k in dataset:
                data1.append(i)
                data2.append(j)
                data3.append(k)

        self.data1 = np.array(data1)
        self.data2 = np.array(data2)
        self.data3 = np.array(data3)

    def __len__(self):
        return len(self.data1)
    
    def __getitem__(self, index):
        return (self.data1[index], self.data2[index], self.data3[index])


user = "Sparsh" #"Parth" "Theo" "Rama"
if user=="Sparsh":
    path = "../../"

trainingData = customDataset(path,
                             ["2021-07-22-1627004019-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f.hdf5",
                              "2021-06-04-1622862148-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f.hdf5"])

In [69]:
len(trainingData)

3490549

In [56]:
'''
MODIFY THIS CELL TO INCLUDE YOUR OWN MODEL
'''

class model(nn.Module):
  def __init__(self):
    super(model, self).__init__()
  def forward(self, x):
    return x

myModel = model()

# Training and Evaluation Loops

In [ ]:
def trainingLoop(modeltoTrain, trainData, lossFunc, optimizer, EPOCHS, BATCH_SIZE, preprocessing = lambda x: x):
  modeltoTrain.train()
  for current_epoch in range(EPOCHS):
      for batch_inputs, actualClasses in iter(torch.utils.data.DataLoader(trainData, batch_size = BATCH_SIZE, shuffle = True)):
          predicted_outputs = modeltoTrain(preprocessing(batch_inputs))
          currentLoss = lossFunc(predicted_outputs, actualClasses)
          currentLoss.backward()
          optimizer.step()
          optimizer.zero_grad()

def evaluationLoop(modeltoTest, trainData, BATCH_SIZE, preprocessing = lambda x: x, postprocessing = lambda x: x):
  modeltoTest.eval()

  allPreds = torch.Tensor([])
  allAnswers = torch.Tensor([])

  for current_epoch in range(EPOCHS):
      for batch_inputs, actualClasses in iter(torch.utils.data.DataLoader(trainData, batch_size = BATCH_SIZE, shuffle = True)):
          predicted_outputs = modeltoTest(preprocessing(batch_inputs))
          allPreds = torch.cat([allPreds, postprocessing(predicted_outputs)])
          allAnswers = torch.cat([allAnswers, actualClasses])

  return allPreds, allAnswers

In [ ]:
EPOCHS = 5
BATCH_SIZE = 64

predictions_before_training, answers_before_training = evaluationLoop(myModel,
                                                                      testingData,
                                                                      BATCH_SIZE,
                                                                      customDataset.preprocessing,
                                                                      customDataset.postprocessing)

trainingLoop(myModel,
             trainingData,
             nn.CrossEntropyLoss(),
             optim.Adam(myDecoder.parameters(), lr = 0.001),
             EPOCHS,
             BATCH_SIZE,
             customDataset.preprocessing)

predictions_after_training, answers_after_training = evaluationLoop(myModel,
                                                                    testingData,
                                                                    BATCH_SIZE,
                                                                    customDataset.preprocessing,
                                                                    customDataset.postprocessing)

# Understanding How Well the Model Is Performing

In [ ]:
print("Before training the model...")
plt.imshow(metrics.confusion_matrix(predictions_before_training, answers_before_training))
plt.show()
print("After training the model...")
plt.imshow(metrics.confusion_matrix(predictions_after_training, answers_after_training))
plt.show()

In [ ]:
import random
placeInBatch = random.randint(0, BATCH_SIZE-1)
images, classes = iter(testDataLoader).__next__()
image = images[placeInBatch]
trueClass = classes[placeInBatch]

print("Sample image:")
plt.imshow(image[0], cmap = "gray")
plt.show()
print("True class:", trueClass.item())
print("Prediction:",
      customDataset.postprocessing
        (
            model(
                  customDataset.preprocessing(image)
                 )
        )
    )

# Saving and Loading Weights

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
secondModel = model()

secondModel.load_state_dict(torch.load("model.pth"))
secondModel.to(device)
secondModel.eval()

In [ ]:
import random
placeInBatch = random.randint(0, BATCH_SIZE-1)
images, classes = iter(testDataLoader).__next__()
image = images[placeInBatch]
trueClass = classes[placeInBatch]

print("Sample image:")
plt.imshow(image[0], cmap = "gray")
plt.show()
print("True class:", trueClass.item())
print("Prediction:",
      customDataset.postprocessing
        (
          secondModel(
                      customDataset.preprocessing(image)
                     )
        )
    )